# Reinforcement Learning on Shower Temperature Adjustment Problem

Problem Statement: During showering, it is very difficult to adjust the optimal shower temperature, sometimes it rather hot and sometime it is rather cold. The objective of this model is to design a system which would automatically adjust the shower temperature between a sweet spot of 37 to 39 degrees. \
It is assumed that the person will be taking the shower for 60 seconds. We use Reinforcement Learning to acheive this objective. The RL Agent is rewarded it is able to maintain a suitable temperature within the range and penalized otherwise.

There are 3 states of operation of the shower knob: \
DOWN (Temperature = 37): State 0 \
MID  (Temperature = 38): State 1 \
UP   (Temperature = 39): State 2

We aslo use some random noise fluctations in the environment to emulate real world like situation where the atmospheric and ventillation conditions of the bathroom play a role.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/RL Shower

/content/drive/MyDrive/Colab Notebooks/RL Shower


In [3]:
pwd

'/content/drive/MyDrive/Colab Notebooks/RL Shower'

In [4]:
ls

'RL Shower Temperature.ipynb'


In [5]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     |████████████████████████████████| 320.4 MB 46 kB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 45.7 MB/s 
     |████████████████████████████████| 459 kB 41.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
 

     |████████████████████████████████| 52 kB 626 kB/s 


In [6]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

## Create Shower Environment

In [7]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [8]:
env = ShowerEnv()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [17]:
env.action_space.sample()

1

In [18]:
env.observation_space.sample()

array([71.2876], dtype=float32)

In [19]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-44
Episode:2 Score:-22
Episode:3 Score:-52
Episode:4 Score:-40
Episode:5 Score:-48
Episode:6 Score:-52
Episode:7 Score:-56
Episode:8 Score:-50
Episode:9 Score:-36
Episode:10 Score:-48


# Creating Deep Learning Model using Keras

In [20]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [21]:
states = env.observation_space.shape
actions = env.action_space.n

In [22]:
print(states)
print(actions)

(1,)
3


In [50]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states, name="input"))
    model.add(Dense(48, activation="relu", name="layer1"))
    model.add(Dense(48, activation="relu", name="layer2"))
    model.add(Dense(48, activation="relu", name="layer3"))
    model.add(Dense(actions, activation='linear', name="output"))
    return model

In [47]:
del model

In [51]:
model = build_model(states, actions)

In [52]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 24)                48        
_________________________________________________________________
layer1 (Dense)               (None, 48)                1200      
_________________________________________________________________
layer2 (Dense)               (None, 48)                2352      
_________________________________________________________________
layer3 (Dense)               (None, 48)                2352      
_________________________________________________________________
output (Dense)               (None, 3)                 147       
Total params: 6,099
Trainable params: 6,099
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-RL

In [53]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [54]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=500000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [55]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=500000, visualize=False, verbose=1)

Training for 500000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 13:14 - reward: 1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 128s 13ms/step - reward: -0.5930
166 episodes - episode_reward: -35.482 [-60.000, 20.000] - loss: 1.500 - mae: 7.749 - mean_q: -10.936

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 128s 13ms/step - reward: 0.2642
167 episodes - episode_reward: 15.533 [-60.000, 56.000] - loss: 1.896 - mae: 7.725 - mean_q: -2.113

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 127s 13ms/step - reward: 0.5712
167 episodes - episode_reward: 34.251 [-24.000, 56.000] - loss: 8.406 - mae: 20.269 - mean_q: 30.887

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 127s 13ms/step - reward: 0.5414
166 episodes - episode_reward: 32.554 [-12.000, 56.000] - loss: 11.015 - mae: 23.525 - mean_q: 35.665

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 127s 13ms/step - reward: 0.5288
167 episodes - episode_reward: 31.677 [-14.000, 58.000]

In [56]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 58.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 54.000, steps: 60
Episode 6: reward: 56.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 56.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 58.000, steps: 60
Episode 26: reward: 60.000, st

In [59]:
dqn.save_weights('models/rlshower_weights.h5f', overwrite=True)

In [60]:
del model
del dqn
del env